# Capstone Project - The Battle of the Neighborhoods (Week 1)

## Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem 

The aim of the project is to find places in South Mumbai, India based on their rating, reviews and prices. As we show you various venus of the busiest city in India and lots of places to hangout, to help visitors select the restuarents based on their interests.

The main purpose of this Project is to summarize a report, focusing to find out the important
reasons that influence consumers preferences and choices related to specialty cuisine restaurants
in South Mumbai. Consumers need information to make their decision in regards to purchasing or treating themselves with a good meal.

Here, we'll identify places that are fit for various individuals based on the information collected from the two APIs and Data Science. Once we have the plot with the venues, any consumer/company can launch an application using the same data and suggest users such information.

## Gathering the Data

To begin with the data injestion, Lets look at the map of South Mumbai. We will collect the data using 2 api's 1.Zomato Api 2.FourSquare Api. 
As FourSquare does not have musch influence in India, We shall combine it with Zomato Api to get a wide range of data.

In [3]:
MUM_LATITUDE = 18.9600
MUM_LONGITUDE = 72.8200
print('The geograpical coordinates of South Mumbai are {}, {}.'.format(MUM_LATITUDE, MUM_LONGITUDE))

The geograpical coordinates of South Mumbai are 18.96, 72.82.


In [4]:
FOURSQUARE_CLIENT_ID = 'IXKO3K1TTUTWKRPUOQQ4DSGQ02BD5PZCDWEKAT5ZIEJHNDQI'
FOURSQUARE_CLIENT_SECRET = '0TN15EHTFJWD3W2HHDUQEME4TWRHF4I1S0G0LICJCBXJR3I4'
RADIUS = 6000 # 4 Km
NO_OF_VENUES = 100
VERSION = '20210114' # Current date

In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
import requests

pd.set_option('display.max_rows', None)

offset = 0
total_venues = 0
foursquare_venues = pd.DataFrame(columns = ['name', 'categories', 'lat', 'lng'])

while (True):
    url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
           '&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}').format(FOURSQUARE_CLIENT_ID, 
                                                                        FOURSQUARE_CLIENT_SECRET, 
                                                                        VERSION, 
                                                                        MUM_LATITUDE, 
                                                                        MUM_LONGITUDE,
                                                                        RADIUS,
                                                                        NO_OF_VENUES,
                                                                        offset)
    result = requests.get(url).json()
    venues_fetched = len(result['response']['groups'][0]['items'])
    total_venues = total_venues + venues_fetched
    print("Total {} venues fetched within a total radius of {} Km".format(venues_fetched, RADIUS/1000))

    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)

    # Filter the columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    venues = venues.loc[:, filtered_columns]

    # Filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

    # Clean all column names
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    foursquare_venues = pd.concat([foursquare_venues, venues], axis = 0, sort = False)
    
    if (venues_fetched < 100):
        break
    else:
        offset = offset + 100

foursquare_venues = foursquare_venues.reset_index(drop = True)
print("\nTotal {} venues fetched".format(total_venues))

Total 100 venues fetched within a total radius of 6.0 Km


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:33: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Total 69 venues fetched within a total radius of 6.0 Km

Total 169 venues fetched


In [8]:
headers = {'user-key': '78fac6501654db5ed2c426dad0821d2c'}
venues_information = []

for index, row in foursquare_venues.iterrows():
    print("Fetching data for venue: {}".format(index + 1))
    venue = []
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=1&lat={}&lon={}&sort=real_distance').format(row['name'], row['lat'], row['lng'])
    result = requests.get(url, headers = headers).json()
    if (len(result['restaurants']) > 0):
        venue.append(result['restaurants'][0]['restaurant']['name'])
        venue.append(result['restaurants'][0]['restaurant']['location']['latitude'])
        venue.append(result['restaurants'][0]['restaurant']['location']['longitude'])
        venue.append(result['restaurants'][0]['restaurant']['average_cost_for_two'])
        venue.append(result['restaurants'][0]['restaurant']['price_range'])
        venue.append(result['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
        venue.append(result['restaurants'][0]['restaurant']['location']['address'])
        venues_information.append(venue)
    else:
        venues_information.append(np.zeros(6))
    
zomato_venues = pd.DataFrame(venues_information, 
                                  columns = ['venue', 'latitude', 
                                             'longitude', 'price_for_two', 
                                             'price_range', 'rating', 'address'])

Fetching data for venue: 1
Fetching data for venue: 2
Fetching data for venue: 3
Fetching data for venue: 4
Fetching data for venue: 5
Fetching data for venue: 6
Fetching data for venue: 7
Fetching data for venue: 8
Fetching data for venue: 9
Fetching data for venue: 10
Fetching data for venue: 11
Fetching data for venue: 12
Fetching data for venue: 13
Fetching data for venue: 14
Fetching data for venue: 15
Fetching data for venue: 16
Fetching data for venue: 17
Fetching data for venue: 18
Fetching data for venue: 19
Fetching data for venue: 20
Fetching data for venue: 21
Fetching data for venue: 22
Fetching data for venue: 23
Fetching data for venue: 24
Fetching data for venue: 25
Fetching data for venue: 26
Fetching data for venue: 27
Fetching data for venue: 28
Fetching data for venue: 29
Fetching data for venue: 30
Fetching data for venue: 31
Fetching data for venue: 32
Fetching data for venue: 33
Fetching data for venue: 34
Fetching data for venue: 35
Fetching data for venue: 36
F

In [9]:
zomato_venues

,venue,latitude,longitude,price_for_two,price_range,rating,address
0,Jaffer Bhai's Delhi Darbar,18.9614636427,72.8233058006,1000.0,3.0,4.0,"195/197, Patthe Bapu Rao Marg, Grant Road East..."
1,Swati Snacks,18.9666012618,72.8136059269,450.0,1.0,4.4,"248, Karai Estate, Opposite Bhatia Hospital, T..."
2,Bachelorr's,18.9541845230,72.8153356165,500.0,2.0,3.8,"45, Chowpatty Sea Face, Netaji Subhash Chandra..."
3,TNBT : The Next Big Thing,18.9563927718,72.8121092543,850.0,2.0,3.9,"29/A, Jethabhai Govindji Building, Opposite Me..."
4,Shree Thaker Bhojanalay,18.9512018821,72.8282779455,1300.0,3.0,4.9,"31, Dadisheth Agyari Lane, Off Kalbadevi Road,..."
5,Jaffer Bhai's Delhi Darbar,18.9614636427,72.8233058006,1000.0,3.0,4.0,"195/197, Patthe Bapu Rao Marg, Grant Road East..."
6,Symphony Paan House,18.9632890804,72.8101294488,100.0,1.0,0,"Shop 2, Sheredon House, August Kranti Marg, Ke..."
7,Soam,18.9574423457,72.8089221194,800.0,2.0,4.5,"Plot-15, 2, Ground Floor, Sadguru Sadan, Babul..."
8,Doolally Taproom,19.0690475172,72.8396464884,1800.0,3.0,4.4,"Ground Floor, Raj Kutir, 3rd Road, Near Railwa..."
9,Sukh Sagar Juice,18.9560842403,72.8134171665,350.0,1.0,3.9,"542 A, Gandhi Building, SVP Road, Near Chowpat..."


### Cleaning the data and dropping the venues which is showing as 0

In [25]:
Zomato_Venues_New = zomato_venues[zomato_venues.venue != 0]

Zomato_Venues_New = Zomato_Venues_New.reset_index(drop=True)

In [26]:
Zomato_Venues_New

,venue,latitude,longitude,price_for_two,price_range,rating,address
0,Jaffer Bhai's Delhi Darbar,18.9614636427,72.8233058006,1000.0,3.0,4.0,"195/197, Patthe Bapu Rao Marg, Grant Road East..."
1,Swati Snacks,18.9666012618,72.8136059269,450.0,1.0,4.4,"248, Karai Estate, Opposite Bhatia Hospital, T..."
2,Bachelorr's,18.9541845230,72.8153356165,500.0,2.0,3.8,"45, Chowpatty Sea Face, Netaji Subhash Chandra..."
3,TNBT : The Next Big Thing,18.9563927718,72.8121092543,850.0,2.0,3.9,"29/A, Jethabhai Govindji Building, Opposite Me..."
4,Shree Thaker Bhojanalay,18.9512018821,72.8282779455,1300.0,3.0,4.9,"31, Dadisheth Agyari Lane, Off Kalbadevi Road,..."
5,Jaffer Bhai's Delhi Darbar,18.9614636427,72.8233058006,1000.0,3.0,4.0,"195/197, Patthe Bapu Rao Marg, Grant Road East..."
6,Symphony Paan House,18.9632890804,72.8101294488,100.0,1.0,0,"Shop 2, Sheredon House, August Kranti Marg, Ke..."
7,Soam,18.9574423457,72.8089221194,800.0,2.0,4.5,"Plot-15, 2, Ground Floor, Sadguru Sadan, Babul..."
8,Doolally Taproom,19.0690475172,72.8396464884,1800.0,3.0,4.4,"Ground Floor, Raj Kutir, 3rd Road, Near Railwa..."
9,Sukh Sagar Juice,18.9560842403,72.8134171665,350.0,1.0,3.9,"542 A, Gandhi Building, SVP Road, Near Chowpat..."
